In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

from sklearn.metrics import(
    classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
)
from sklearn.preprocessing import (
    StandardScaler, LabelBinarizer, FunctionTransformer,PolynomialFeatures, OrdinalEncoder,LabelEncoder,MinMaxScaler
)
# from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.pipeline import Pipeline, FeatureUnion

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
from keras.utils import np_utils

In [ ]:
import mitosheet

#### For Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Ms Imumisen/smart_home_ds.csv")

#### For Jupyter Lab

In [ ]:
data= pd.read_csv('D:\\Desktop Files\\weka data\\Ms Imumu\\hh130\\hh130.ann.features.csv')

In [ ]:
mitosheet.sheet(data)

MitoWidget(analysis_name='UUID-716c6500-92c2-499a-8c65-195f4a60998b', code_json='{"imports": "from mitosheet i…

In [ ]:
# MITO CODE START (DO NOT EDIT)

from mitosheet import * # Import necessary functions from Mito
register_analysis('UUID-716c6500-92c2-499a-8c65-195f4a60998b') # Let Mito know which analysis is being run

# Pivoted data into df2
df2 = pd.DataFrame(data={})

  
# MITO CODE END (DO NOT EDIT)

#### Data Wrangling and Processing

In [ ]:
data.head(4)

,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation,...,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea,activity
0,2.0,7371.0,3.0,5722.0,1.0,0.0,0.0,1.0,1.0,1.0,...,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep
1,2.0,7373.0,3.0,5723.0,2.0,0.0,0.0,1.0,1.0,1.0,...,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep
2,2.0,7374.0,3.0,5496.0,1.0,0.0,0.0,1.0,1.0,1.0,...,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep
3,2.0,7375.0,3.0,5496.0,1.0,0.0,0.0,1.0,1.0,1.0,...,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,86400.0,Sleep


In [ ]:
X_df = data.drop(labels=['activity'], axis=1)
y_df = data['activity']

In [ ]:
y_df

In [ ]:
data.corr()

In [ ]:
data.corr().info()

In [ ]:
corr_data = data.corr()

#### Filtering Columns with Contant Values

In [ ]:
# Execute the following script to create a filter for constant features.
constant_filter = VarianceThreshold(threshold=0)
# Next, we need to simply apply this filter to our training set as shown in the following example:
constant_filter.fit(X_df)

VarianceThreshold(threshold=0)

In [ ]:
# Execute the following script to see the number of non-constant features.

len(X_df.columns[constant_filter.get_support()])

25

In [ ]:
# List of contants column
constant_columns = [column for column in X_df.columns
                    if column not in X_df.columns[constant_filter.get_support()]]

In [ ]:
constant_columns

['numDistinctSensors',
 'sensorCount-Chair',
 'sensorCount-DiningRoom',
 'sensorCount-Hall',
 'sensorCount-Office',
 'sensorCount-WorkArea',
 'sensorElTime-Chair',
 'sensorElTime-DiningRoom',
 'sensorElTime-Hall',
 'sensorElTime-Office',
 'sensorElTime-WorkArea']

In [ ]:
X_df.head(2)

,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation,complexity,activityChange,areaTransitions,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-OutsideDoor,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-OutsideDoor
0,2.0,7371.0,3.0,5722.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.498951,0.0,0.0,34.35,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,86400.0
1,2.0,7373.0,3.0,5723.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.710117,0.0,0.0,34.35,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,86400.0


In [ ]:
# Finally, to remove constant features from training and test sets, we can use the transform() method of the constant_filter. 
# Execute the following script to do so:

# X_filtered = constant_filter.transform(X_df)
# test_features = constant_filter.transform(test_features)
# X_filtered.shape

# Executing this and skip transform if you want to filter quasi-constant before transform
X_df.drop(labels=constant_columns, axis=1, inplace=True)

In [ ]:
X_df

#### Filtering Out Of Contant-filteres dataset quasi-constant

In [ ]:
# Let's create our quasi-constant filter. Execute the following script to do so:
qconstant_filter = VarianceThreshold(threshold=0.01)
qconstant_filter.fit(X_df)

VarianceThreshold(threshold=0.01)

In [ ]:
len(X_df.columns[qconstant_filter.get_support()])

25

In [ ]:
qconstant_columns = [column for column in X_df.columns
                    if column not in X_df.columns[qconstant_filter.get_support()]]

print(len(qconstant_columns))

0


In [ ]:
for column in qconstant_columns:
    print(column)

In [ ]:
train_features = qconstant_filter.transform(train_features)
# test_features = qconstant_filter.transform(test_features)

train_features.shape, test_features.shape

#### Removing Duplicate Features

In [ ]:
# We will use this method to first take a transpose of our dataset as shown below:
X_T = X_df.T
X_T.shape

(25, 157372)

In [ ]:
# Let's find the total number of duplicate features in our dataset using the sum() method, chained with the duplicated() method as shown below.
print(X_T.duplicated().sum())

1


In [ ]:
# X_T.head(2)
unique_features.head(2)

,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastMotionLocation,complexity,activityChange,areaTransitions,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-OutsideDoor,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-OutsideDoor
0,2.0,7371.0,3.0,5722.0,1.0,0.0,0.0,1.0,1.0,0.0,0.498951,0.0,0.0,34.35,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,86400.0
1,2.0,7373.0,3.0,5723.0,2.0,0.0,0.0,1.0,1.0,0.0,0.710117,0.0,0.0,34.35,0.0,0.0,0.0,0.0,86400.0,0.0,86400.0,86400.0,86400.0,86400.0


In [ ]:
unique_features = X_T.drop_duplicates(keep='first').T
# bNow, let's print the shape of our new training set without duplicate features:
unique_features.shape

(157372, 24)

In [ ]:
duplicated_features = [dup_col for dup_col in X_df.columns if dup_col not in unique_features.columns]
duplicated_features

['lastSensorLocation']

#### Removing Correlated Features

In [ ]:
# Execute the following script, to remove non-numeric features from the dataset.
num_colums = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = list(unique_features.select_dtypes(include=num_colums).columns)
unique_numeric_features = unique_features[numerical_columns]

In [ ]:
# Let's first create correlation matrix for the columns in the dataset and an empty set that will contain all the correlated features. 
# Execute the following script to do so:
correlated_features = set()
correlation_matrix = unique_numeric_features.corr()

In [ ]:
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [ ]:
# len(correlated_features)
print(correlated_features)

{'lastSensorEventSeconds', 'lastMotionLocation', 'sensorElTime-OutsideDoor'}


In [ ]:
# The following script removes these columns from the dataset:
unique_numeric_features.drop(labels=correlated_features, axis=1, inplace=True)

In [ ]:
unique_numeric_features.shape

(157372, 21)

In [ ]:
# convert dataframe to numpy array
values = data.values
X, y = values[:, 0:36], values[:,36]
# X = X.reshape(len(X), 36, 1)

X = np.asarray(X).astype(np.float64)

In [ ]:
encoder = LabelEncoder()
encoder.fit(y_df)
encoded_Y = encoder.transform(y_df)

In [ ]:
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(unique_numeric_features, encoded_Y,test_size=0.3,random_state=42)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,test_size=0.5,random_state=42)

In [ ]:
sc = StandardScaler()
S_train = sc.fit_transform(X_train)
S_val = sc.transform(X_val)

In [ ]:
X_train.shape,S_train.shape,y_train.shape

((110160, 21), (110160, 21), (110160,))

In [ ]:
# pipe1a = Pipeline([
#     ('map', mapper),
# ])

In [ ]:
# multi-class classification with Keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=21, activation='relu'))
    model.add(Dense(40, activation='relu'))
    # model.add(Dense(30, activation='relu'))
    # model.add(Dense(5, activation='relu'))
    model.add(Dense(28, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=100, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, S_train, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# _, accuracy = model.evaluate(x, y)
# print("Model accuracy: %.2f"% (accuracy*100))

Epoch 1/10
992/992 [==============================] - 2s 1ms/step - loss: 1.2618 - accuracy: 0.5865
Epoch 2/10
992/992 [==============================] - 1s 1ms/step - loss: 0.9973 - accuracy: 0.6439
Epoch 3/10
992/992 [==============================] - 1s 1ms/step - loss: 0.9384 - accuracy: 0.6625
Epoch 4/10
992/992 [==============================] - 1s 1ms/step - loss: 0.9040 - accuracy: 0.6745
Epoch 5/10
992/992 [==============================] - 1s 1ms/step - loss: 0.8778 - accuracy: 0.6804
Epoch 6/10
992/992 [==============================] - 1s 1ms/step - loss: 0.8548 - accuracy: 0.6890
Epoch 7/10
992/992 [==============================] - 1s 1ms/step - loss: 0.8362 - accuracy: 0.6950
Epoch 8/10
992/992 [==============================] - 1s 1ms/step - loss: 0.8188 - accuracy: 0.7018
Epoch 9/10
992/992 [==============================] - 1s 1ms/step - loss: 0.8033 - accuracy: 0.7071
Epoch 10/10
111/111 [==============================] - 0s 1ms/step - loss: 0.7812 - accuracy: 0.7099

In [ ]:
from google.colab import files
unique_numeric_features.to_csv('unique_numeric_features.csv') 
files.download("unique_numeric_features.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>